## 1.3 CounterApp

Przejdźmy do utworzenia pierwszej aplikacji i stwórzmy prosty licznik. 

<img src="https://media0.giphy.com/media/Lh9I3cXP2VdH1sFrsc/giphy.gif?cid=790b7611d0829bffb2f61d086d037fcf87f9b0f805ef3b33&rid=giphy.gif&ct=g" width="200" />

Rozpoczniemy od utworzenia interfejsu zawierającego jeden `Button` i pole `TextView`. Przechodzimy do pliku `activity_main.xml`.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <TextView
        android:id="@+id/show_count"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:text="0"
        android:textSize="250sp"
        android:gravity="center"
        android:textStyle="bold"/>

    <Button
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="Count Up"
        android:onClick="countUpButton"/>
</LinearLayout>

Zauważmy kilka nowych elementów:
- `android:id` - jest identyfikatorem przez który możemy odnosić się do tego elementu w kodzie
- `gravity` - wyśrodkowanie elementu
- `android:onClick` - jedna z wielu możliwości dodania metody obsługi zdarzenia `onClick` - metoda nosi nazwę **countUpButton**, jej implementacja będzie znajdować się w pliku głównej aktywności

Przejdźmy teraz do pliku `MainActivity.kt` w którym implementujemy funckjonalność głównej aktywności i dodajmy obsługę zdarzenia `onClick`. W pierwszej kolejności musimy utworzyć pole reprezentujące `TextView` - będzie to pole niemutowalne. Musimy również powiązać pole `showCount` z elementem layoutu `show_count`, robimy to wywołując metodę `findViewById` - jest to jedna z wielu dostępnym opcji. Ponieważ nash `showCount` jest niemutowalny inicjalizację wykonujemy przez delegat `by lazy`.

In [ ]:
private val showCount: TextView by lazy{findViewById(R.id.show_count)}

Następnie dodajmy zmienną reprezentującą stan licznika

In [ ]:
private var count = 0

Kolejnym krokiem jest wyświetlenie tekstu, w tym celu wywołujemy metodę `setText`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)

    showCount.text = String.format(Locale.GERMAN, "%,d", count)
}

Ostatnim krokiem jest dodanie implementacji metody `countUpButton`. W ciele klasy `MainActivity` dodajemy

In [ ]:
fun countUpButton(view: View?) {
    count++
    showCount.text = count.toString()
}

Na tym etapie możemy zbudować projekt. Pełny kod źródłowy:

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <TextView
        android:id="@+id/show_count"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:text="0"
        android:textSize="250sp"
        android:gravity="center"
        android:textStyle="bold"/>

    <Button
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="Count Up"
        android:onClick="countUpButton"/>
</LinearLayout>

`MainActivity.kt`

In [ ]:
package pl.edu.uwr.pum.counterappkotlin

import android.os.Bundle
import android.view.View
import android.widget.TextView
import androidx.appcompat.app.AppCompatActivity
import java.util.*

class MainActivity : AppCompatActivity() {

    private var count = 0
    private val showCount: TextView by lazy{findViewById(R.id.show_count)}

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        showCount.text = String.format(Locale.GERMAN, "%,d", count)
    }

    fun countUpButton(view: View?) {
        count++
        showCount.text = count.toString()
    }
}

<img src="https://fv9-6.failiem.lv/thumb_show.php?i=e73zs4cbf&view" width="200" />

Zwróćmy uwagę że przy zmianie orientacji urządzenia stan licznika jest resetowany

<img src="https://fv9-5.failiem.lv/thumb_show.php?i=auy6zck7f&view" width="200" />

 Jest to związane z cyklem życia aktywności - aktywność jest niszczona i odtwarzana przy każdej zmianie konfiguracji. Możemy dane zachować przy pomocy obiektu typu `Bundle`. Zwróćmy uwagę na sygnaturę metody `OnCreate(savedInstanceState: Bundle?)` - przyjmuje ona jeden argument, to właśnie w nim możemy przekazać dane których nie chcemy utracić. Jest to typ zerowalny, ponieważ przy pierwszym uruchomieniu i wywołaniu metody `OnCreate` obiekt `Bundle` zawsze będzie miał wartość `null`.

W pierwszym kroku musime zapisać dane które chcemy zachować i spakować je do obiektu `Bundle`. W tym celu mamy specjalną metodę którą możemy nadpisać w klasie `MainActivity` - metoda nazywa się `onSaveInstanceState`.

In [ ]:
override fun onSaveInstanceState(outState: Bundle) {
    super.onSaveInstanceState(outState)
    outState.putInt("counter_state", count)
}

Do obiektu typu `Bundle` (standardowo nazywanym `outState` w metodzie `onSaveInstanceState`) możemy dodać dowolną liczbę danych za pomocą odpowiednich metod, tutaj chcemy przekazać stan licznika, więc używamy metody `putInt`. Metoda ta przyjmuje dwa argumenty:
- klucz typu `String` który jest unikalnym identyfikatorem
- wartość typu odpowiadającemu wywoływnej metodzie - tutaj wykorzystując metodę `putInt` wartość będzie typu `int`

Za każdym razem przy zmianie konfiguracji urządzenia metoda `onSaveInstanceState` jest wywoływana automatycznie. Dane możemy odebrać w metodzie `OnCreate`, która przyjmuje argueemnt typu `Bundle` (nie jest to jedyna możliwość - jest również metoda `onRestoreinstanceState`). Aby rozpakować przekazane dane do zmiennej `count` przypisujemy wartość wyciągniętą z argumentu `savedInstanceState` - pamiętając o sprawdzeniu czy `Bundle` nie ma wartości `null`. Posiada ją zawsze przy pierwszym uruchomieniu. Do metody `OnCreate` dodajemy kod

In [ ]:
if (savedInstanceState != null)
    count = savedInstanceState.getInt("counter_state")

Tutaj istotna jest kolejność wywołań - wpierw próbujemy przypisać do `count` wartość z `savedInstanceState`, jeżeli to się nie uda wartoość `count` pozostaje domyślna. Wartość wyjmujemy za pomocą analogicznej do `putInt` metody `getInt`, która przyjmuje jeden argument - jest to unikalny. Oczywistym jest że klucz w metodach `putInt` i `getInt` musi być ten sam. Dopiero wtedy wykonujemy aktualizację pola `showCount` (**TextView**). Pełny kod metody `OnCreate`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)

    if (savedInstanceState != null)
        count = savedInstanceState.getInt("counter_state")

    showCount.text = String.format(Locale.GERMAN, "%,d", count)
}

Przy prawidłowej implementacji stan licznika powinien być zachowany w przypadku wystąpienia zdarzenia rotacji.

Pałny kod `MainActivity.kt`

In [ ]:
package pl.edu.uwr.pum.counterappkotlin

import android.os.Bundle
import android.view.View
import android.widget.TextView
import androidx.appcompat.app.AppCompatActivity
import java.util.*

class MainActivity : AppCompatActivity() {

    private var count: Int = 0
    private val showCount: TextView by lazy{findViewById(R.id.show_count)}

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        if (savedInstanceState != null)
            count = savedInstanceState.getInt("counter_state")


        showCount.text = String.format(Locale.GERMAN, "%,d", count)
    }

    fun countUpButton(view: View?) {
        count++
        showCount.text = count.toString()
    }

    override fun onSaveInstanceState(outState: Bundle) {
        super.onSaveInstanceState(outState)
        outState.putInt("counter_state", count)
    }
}